In [ ]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import keras

In [18]:
train_ds, val_ds, test_ds = tfds.load(
    'wider_face',
    split=['train', 'validation', 'test'],
    shuffle_files=True
)

In [21]:
def preprocess(sample):
    img=tf.image.resize(sample['image'],(128,128))
    img=img/255.0
    box=sample['faces']['bbox']
    return img, box

train_ds=train_ds.map(preprocess).cache().batch(16).shuffle(100).prefetch(tf.data.AUTOTUNE)
val_ds=val_ds.map(preprocess).cache().batch(16)
test_ds=test_ds.map(preprocess).cache().batch(16)

In [ ]:
#từ 1 tới 5 theo mô hình vgg16, bỏ phần dưới từ pooling layer ở block 5 đến hết cấu trúc vgg16 gốc
#từ 6 đến 11 là phần layer thêm vào theo cấu trúc ssd300
model=keras.Sequential([
    #1
    keras.layers.Conv2D(64,input_shape=(300,300,3),activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.Conv2D(64,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.MaxPool2D(pool_size=(2,2),strides=2),
    #2
    keras.layers.Conv2D(128,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.Conv2D(128,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.MaxPool2D(pool_size=(2,2),strides=2),
    #3
    keras.layers.Conv2D(256,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.Conv2D(256,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.Conv2D(256,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.MaxPool2D(pool_size=(2,2),strides=2),
    #4
    keras.layers.Conv2D(512,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.Conv2D(512,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.Conv2D(512,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.MaxPool2D(pool_size=(2,2),strides=2),
    #5
    keras.layers.Conv2D(512,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.Conv2D(512,activation='relu',padding='same',kernel_size=(3, 3)),
    keras.layers.Conv2D(512,activation='relu',padding='same',kernel_size=(3, 3)),
    #6
    keras.layers.Conv2D(1024,activation='relu',dilation_rate=6,padding='same',kernel_size=(3, 3)),
    #7
    keras.layers.Conv2D(1024,activation='relu',padding='same',kernel_size=(1, 1)),
    #8
    keras.layers.Conv2D(256,activation='relu',padding='same',kernel_size=(1, 1)),
    keras.layers.Conv2D(512,activation='relu',padding='same',kernel_size=(3, 3),strides=2),
    #9
    keras.layers.Conv2D(128,activation='relu',padding='same',kernel_size=(1, 1)),
    keras.layers.Conv2D(256,activation='relu',padding='same',kernel_size=(3, 3),strides=2),
    #10
    keras.layers.Conv2D(128,activation='relu',padding='same',kernel_size=(1, 1)),
    keras.layers.Conv2D(256,activation='relu',padding='valid',kernel_size=(3, 3),strides=1),
    #11
    keras.layers.Conv2D(128,activation='relu',padding='valid',kernel_size=(1, 1)),
    keras.layers.Conv2D(256,activation='relu',padding='valid',kernel_size=(3, 3),strides=1)
])

In [ ]:
# for sample in train_ds.take(9):
#     h, w=sample['image'].shape[:2]
#     img=sample['image'].numpy()
#     for box in sample['faces']['bbox']:
#         ymin, xmin, ymax, xmax=box
#         x=int(xmin*w)
#         y=int(ymin*h)
#         width=int((xmax-xmin)*w)
#         height=int((ymax-ymin)*h)
#         cv2.rectangle(img,(x,y),(x+width,y+height),(0,0,255),2)
#     plt.imshow(img)